# Thu thập dữ liệu

In [ ]:
import requests
from bs4 import BeautifulSoup

## Get tất cả các link laptop

In [ ]:
all_brands_links = [
    'https://phongvu.vn/laptop-asus-1058.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-acer-1059.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-Dell-1056.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/macbook-1061.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-hp-1057.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-lenovo-1060.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-lg-1268.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-msi-1081.cat?pv_source=homepage&pv_medium=de-megamenu-text&page='
]

items = [] # danh sách các link laptop

# parse để lấy link
for brand in all_brands_links:
    for page in range(1, 50):
        pv_laptop_url = brand + str(page)
        while True:
            try:
                html_text = requests.get(pv_laptop_url).text
                break
            except:
                continue
        tree = BeautifulSoup(html_text, 'html.parser')
        found = tree.find_all('a', {'class': 'css-1rhapru'})
        if len(found) == 0:
            break
        items = items + found
    print(brand)

In [ ]:
len(items)

Tổng cộng get được 794 link laptop.

In [ ]:
# xem qua các link
for item in items[:5]:
    print(item['href'])

In [ ]:
import csv

## Parse từng link và ghi ra file csv

File dữ liệu: `data.csv`

In [ ]:
fields = {'Thương hiệu': 'Brand', 'Bảo hành': 'Warranty', 'Màu sắc': 'Color', 'Series laptop': 'SeriesLaptop', 'Part-number': 'PartNum', 'Thế hệ CPU': 'CPUgen', 'CPU': 'CPU', 'Chip đồ họa': 'GraphicChip', 'RAM': 'RAM', 'Màn hình': 'Screen', 'Lưu trữ': 'Storage', 'Số cổng lưu trữ tối đa': 'MaxStoPortNum', 'Kiểu khe M.2 hỗ trợ': 'SupportM2', 'Cổng xuất hình': 'OutVideoPort', 'Cổng kết nối': 'ConnectPort', 'Kết nối không dây': 'Wireless', 'Bàn phím': 'Keyboard', 'Hệ điều hành': 'OS', 'Kích thước': 'Size', 'Pin': 'Pin', 'Khối lượng': 'Weight', 'Đèn LED trên máy': 'LED', 'Phụ kiện đi kèm': 'Accessories', 'Bảo mật': 'Security', 'Ổ đĩa quang': 'OptDrive', 'Tính năng': 'Feature'}

fieldnames = ['SKU', 'Title', 'Price', 'Brand', 'Warranty', 'Color', 'SeriesLaptop', 'PartNum', 'CPUgen', 'CPU', 'GraphicChip', 'RAM', 'Screen', 'Storage', 'MaxStoPortNum', 'SupportM2', 'OutVideoPort', 'ConnectPort', 'Wireless', 'Keyboard', 'OS', 'Size', 'Pin', 'Weight', 'LED', 'Accessories', 'Security', 'OptDrive', 'Feature']

In [ ]:
file = open('data.csv', 'w', encoding='utf-8')
file_writer = csv.DictWriter(file, fieldnames=fieldnames)
file_writer.writeheader()

count = 0
for item in items:
    count += 1
    url = 'https://phongvu.vn' + item['href']
    while True:
        try:
            laptop_html_text = requests.get(url).text
            break
        except:
            continue
    laptop_html_tree = BeautifulSoup(laptop_html_text, 'html.parser')
    title = laptop_html_tree.find('div', {'class': 'css-1jpdzyd'}).text
    sku = laptop_html_tree.find('div', {'class': 'css-5nimvs'}).text.split(' ')[-1]
    price = laptop_html_tree.find('span', {'class': 'css-3725in'}).text[:-1]
    values = {'SKU': sku, 'Title': title, 'Price': price} 
    for i in laptop_html_tree.find_all('div', {'class': 'css-7j9rw7'}):
        name = i.find('span', {'class': 'css-6z2lgz'}).text.strip(' ')
        if name in fields:
            values[fields[name]] = i.find('div', {'class': 'css-111s35w'}).text.strip(' ')
    file_writer.writerow(values)
    if count % 10 == 0:
        print(count)
file.close()